# Homework 9 
## GIS5090
### Carter Hanford
This is the Jupyter notebook for Homework 9 in GIS5090 - Intro to Python for GIS & Remote Sensing. This homework deals with using the ArcGIS API for Python. 

This homework asks us to create a Jupyter notebook which includes the following:
* `Contains a map.`
* `Zooms to a particular city of interest.`
* `Changes the basemap from the default Terrain basemap.`
* `Please use Markdown to apply meaningful descriptions of your steps.`
* `Contains at least 3 layers, not including the basemap.`

In this case, the particular city of interest will be St. Louis city, Missouri. We'll start by creating the map layer. We'll write the map we import to an object called `map1` and as we make changes to the map down the notebook, this original layer will be updated with the changes. 

In [28]:
# import modules
import arcpy
import arcgis 
from arcgis import GIS
from arcgis.features import SpatialDataFrame
from arcgis.features import FeatureLayer

# connect to arcgis online
gis = GIS("http://slustl.maps.arcgis.com/home", "carter.hanford_SLUSTL")

Enter password: ········


In [52]:
# build map
map1 = gis.map('Missouri')
map1.basemap = 'streets-vector'

# zoom to st. louis
location = arcgis.geocoding.geocode('St. Louis, MO', max_locations = 1)[0]
map1.extent = location['extent']
map1

MapView(layout=Layout(height='400px', width='100%'))

Now that we have a map of St. Louis City, we can continue with the analysis and include some content layers. We're already connected to ArcGIS online, so we can search some layers to add to our map.

One of the layers that I want to add to this map are the `census tracts`. I work with census tracts for all of my research, so replicating that here will be useful. 

In [53]:
# search layers
demographic_layers = gis.content.search("USA Demographics", outside_org=True, item_type="Feature Layer")
demographic_layers
#type(demographic_layers[4])

# add census tracts layer & query to St. Louis City
fl = FeatureLayer(demographic_layers[4].layers[0].url, gis = gis)
sub_fl = fl.query("CNTY_FIPS = '510'")
map1.add_layer(sub_fl)

Now that we've got our census tracts layer onto the map, let's add some more content. 

I searched our SLU ArcGIS organization website and found someone who made a map of St. Louis basketball courts. Let's add that feature service to our map:

In [54]:
# retrieve basketball courts feature layer
items = gis.content.search('title:STL_Basketball_Courts owner:abrok_SLU', 'feature layer')
print(items)

# add basketball courts to map
fl_bball = FeatureLayer(items[2].url, gis = gis)
map1.add_layer(fl_bball)

# add basketball courts buffer to map
fl_bball_buffer = FeatureLayer(items[1].url, gis = gis)
map1.add_layer(fl_bball_buffer)

[<Item title:"STL_Basketball_Courts___Basketball_Court_Centroid_Density" type:Feature Layer Collection owner:abrok_SLU>, <Item title:"Buffer_of_STL_Basketball_Courts___Basketball_Court_Centroid__Buffer" type:Feature Layer Collection owner:abrok_SLU>, <Item title:"STL_Basketball_Courts" type:Feature Layer Collection owner:abrok_SLU>]


Now we have three layers on our original map of St. Louis City. First, we added `census tracts` layer and used a query to filter to St. Louis City. Next, we added point data from a feature service which includes basketball courts in St. Louis, and finally, we added buffers which correspond with those courts. 

* Hey Dr. Brunner, any tips for working with symbology using mapview in ArcGIS? I want to change the color schemes of the points and census tracts on this map but not sure how. 

## Part 2 - Geocoding 
Part 2 of this homework assignment has us dealing with the `addresses.csv` folder downloaded from blackboard. We're asked to geocode these addresses, add them to a map, then save the map and compare any commonalities between the addresses.

We'll need some other modules for this problem, so let's load them up and read in the data:

In [96]:
# create new map 
new_map = gis.map("United States")
new_map

MapView(layout=Layout(height='400px', width='100%'))

In [103]:
# import necessary modules
from arcgis.gis import GIS
from arcgis.geocoding import Geocoder, get_geocoders
from arcgis.geocoding import geocode
import pandas as pd

# read in addresses
addresses = ["470 E Lockwood Ave, Webster Groves, MO 63119", 
             "1 Brookings Dr, St. Louis, MO 63130", 
             "1 N Grand Blvd, St. Louis, MO 63103",
             "1 Hairpin Dr, Edwardsville, IL 62026",
             "1263 Lincoln Dr, Carbondale, IL 62901",
             "1 University Blvd, St. Louis, MO 63121", 
             "5000 Holmes St, Kansas City, MO 64110",
             "6823 St Charles Ave, New Orleans, LA 70118",
             "300 College Park, Dayton, OH 45469",
             "2807 N Glebe Rd, Arlington, VA 22207"]

# geocode addresses
for address in addresses:
    home = geocode(address)[0]
    popup = { 
        "title" : "Home", 
        "content" : home['address']
        }
    new_map.draw(home['location'], popup)

These addresses are definiteley universities. I'm really not sure what the correlation is between them, however! At first I thought they were address of schools in the Atlantic 10 conference, but definitely not that either. Maybe they're just random universities.

Now that we have the addresses geocoded and on the map, we can complete the problem by saving our new map as a webmap to arcGIS online. We'll do that below:

In [106]:
# create properties of webmap
webmap_properties = {'title':'Addresses Webmap',
                    'snippet': 'Web map of addresses csv file',
                    'tags':['automation', 'python']}

# save addresses map as webmap
webmap_item = new_map.save(webmap_properties)
webmap_item

<Item title:"Addresses Webmap" type:Web Map owner:carter.hanford_SLUSTL>